In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
from IPython.display import display_html
import warnings

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize


/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
#def display_side_by_side(*args):
    #html_str = ''
    #for df in args:
        #html_str += df.to_html()
    #display_html(html_str.replace(
        #'table', 'table style="display:inline"'), raw=True)



In [3]:
%%time
anime = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/anime.csv')
watching_status = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/watching_status.csv')
liste = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/animelist.csv')
rating=pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/rating_complete.csv')
synopsis = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/anime_with_synopsis.csv')

CPU times: user 47.6 s, sys: 24.4 s, total: 1min 11s
Wall time: 1min 14s


In [4]:
liste.columns

Index(['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes'], dtype='object')

In [5]:
liste['watching_status'].values

array([1, 1, 1, ..., 2, 2, 6])

In [6]:
liste_selected = liste[['user_id', 'anime_id', 'rating']]

In [7]:
import random
liste_selected = liste_selected[liste_selected['user_id'].isin(random.sample(list(liste_selected['user_id'].unique()), 
                                                                                                  k=5000))]

In [8]:
user_anime_interaction = pd.pivot_table(liste_selected, index='user_id', columns='anime_id', values='rating')


In [9]:
user_id = list(user_anime_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [10]:
user_anime_interaction_csr = csr_matrix(user_anime_interaction.values)
user_anime_interaction_csr

<5000x17315 sparse matrix of type '<class 'numpy.float64'>'
	with 85871223 stored elements in Compressed Sparse Row format>

In [11]:
user_anime_interaction = user_anime_interaction.fillna(0)

In [12]:
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(user_anime_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

ValueError: Not all input values are finite. Check the input for NaNs and infinite values.

In [ ]:
item_dict ={}
df = anime[['MAL_ID', 'Name']].sort_values('MAL_ID').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'MAL_ID'])] = df.loc[i,'Name']

In [ ]:
anime.columns

In [ ]:
anime.Score.replace('Unknown', '0', inplace=True)
anime.Score = pd.to_numeric(anime['Score'])
anime['Score'] = anime['Score'].apply(lambda x: round(x*2)/2)
bibtest = anime[['Name', 'Score', 'Genders']]
bibtest= pd.get_dummies(bibtest, columns = ['Name', 'Score', 'Genders'])

In [ ]:
anime[['Name', 'Rating', 'Members', 'Type']].sort_values(by='Rating', ascending=False).query('Members>500000')[:5]

In [ ]:
from collections import defaultdict

all_genres = defaultdict(int)

for genres in anime['Genders']:
    for genre in genres.split(','):
        all_genres[genre.strip()] += 1

In [ ]:
all_genres.keys().nunique()

In [ ]:
watching_status = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/watching_status.csv')

In [ ]:
watching_status.shape

In [ ]:
liste = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/animelist.csv')

In [ ]:
anime['Ranked'][12]

In [ ]:
liste1 = liste.sample(1000000)

In [ ]:
liste_user = liste1.groupby(['user_id'])

In [ ]:
liste_user= pd.DataFrame(liste_user)

In [ ]:
test = liste1[liste1.watched_episodes==0]

In [ ]:
test['user_id'].count()

In [ ]:
liste_user_watched = liste.groupby(['user_id', 'watched_episodes'])

In [ ]:
liste_user_watched.head()

In [ ]:
synopsis = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/anime_with_synopsis.csv')

In [ ]:
synopsis

In [ ]:
rating=pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/rating_complete.csv')

In [ ]:
rating.columns


In [ ]:
print(rating.columns)
print(liste.columns)
print(synopsis.columns)
print(anime.columns)

In [ ]:
boo = set(rating.anime_id.unique())

In [ ]:
doo = set(anime.MAL_ID.unique())

In [ ]:
rating.head()

In [ ]:
anime.loc[anime.MAL_ID.isin([430,1004,3010,570,2762])]

In [ ]:
liste['anime_id']

In [ ]:
anime['MAL_ID'][0]

In [ ]:
synopsis['MAL_ID'][0]

In [ ]:
rating['anime_id']

In [ ]:
user_anime_interaction.reset_index(inplace=True)
liste.reset_index(inplace=True)

In [ ]:
def sample_recommendation_user(model, liste, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    
    n_users, n_items = liste.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=user_anime_interaction_csr))
    scores.index = liste.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(liste.loc[user_id,:] \
                                 [liste.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1

In [ ]:
liste['user_id']

In [ ]:
sample_recommendation_user(model, user_anime_interaction, "1036", user_dict, item_dict)

In [ ]:
user_dict

In [ ]:
liste_selected